In [1]:
from inference_be import *

CUDA is available!  Training on GPU ...


In [2]:
class TransformedDataset(Dataset):
    def __init__(self, dataset: Dataset, transform: transforms.Compose):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        img, label = self.dataset[index]

        if self.transform:
            img = self.transform(img)
        
        return img, label

def top_k_accuracy(output, target, k=5):
    batch_size = target.size(0)
    _, pred = output.topk(k, 1, True, True)  # Get top-k predictions
    pred = pred.t()  # Transpose predictions for comparison
    correct = pred.eq(target.reshape(1, -1).expand_as(pred))  # Compare predictions with target
    correct_k = correct[:k].reshape(-1).float().sum(0, keepdim = True)  # Calculate correct top-k
    return correct_k.mul_(1.0 / batch_size).detach()  # Calculate top-k accuracy

def evaluate(model, loss_fn, data_loader):
    model.eval()

    loss = 0
    correct = 0
    total = 0
    top_1_accuracy = 0
    top_5_accuracy = 0

    progress_bar = tqdm(data_loader, desc = "Validating")

    with torch.no_grad():
        for batchX, batchY in progress_bar:
            batchX, batchY = batchX.to(device), batchY.to(device)

            output = model(batchX)
            predicted_labels = torch.argmax(output, dim = 1)

            loss += loss_fn(output, batchY).detach() * batchX.size(0)
            correct += (predicted_labels == batchY.type(torch.long)).sum().detach()
            total += batchX.size(0)
            top_1_accuracy += top_k_accuracy(output, batchY, k=1) * batchX.size(0)
            top_5_accuracy += top_k_accuracy(output, batchY, k=5) * batchX.size(0)
    
    return loss.item() / total, correct.item() / total, top_1_accuracy.item() / total, top_5_accuracy.item() / total


In [3]:
test_dataset = datasets.ImageFolder(root = "./test_data")

In [4]:
models = [
    "efficientnet_v2_l", "VGG16", "VGG19", "ResNet152", "ResNet18", "EfficientNetB0", "MobileNetV4", "EdgeNeXtXS", "EdgeNeXtS-USI", "EVA02", "TinyViT11M", "TinyViT21M", "TinyViT5M"
]

In [5]:
# NOTE: Input model name *FROM THE LIST ABOVE*
model_name = "efficientnet_v2_l"

# Create model and fetch transforms
model, transforms = create_model(model_name, len(test_dataset.classes))
model.to(device)

# Freeze the model for inference
for param in model.parameters():
    param.requires_grad = False

g:\FAI A4\FIT5047 - A4\Inference Testing\inference_be.py:385: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_warmed_acc_efficient

In [6]:
# Create test loader with transforms
test_loader = DataLoader(TransformedDataset(test_dataset, transforms), batch_size=32, shuffle=False)

# Create loss function
loss_fn = nn.CrossEntropyLoss()

In [7]:
import time

In [11]:
start_time = time.time()
test_loss, test_acc, top_1, top_5 = evaluate(model, loss_fn, test_loader)
print(f'Test Loss: {test_loss:.4f} - Test Accuracy: {test_acc*100:.4f}% - Top 1 Accuracy: {top_1} - Top 5 Accuracy: {top_5}')
print(f'Total inference time (including transformation): {time.time() - start_time:.2f} seconds')

Validating: 100%|██████████| 32/32 [00:06<00:00,  4.92it/s]

Test Loss: 0.4783 - Test Accuracy: 85.7424% - Top 1 Accuracy: 0.8574237954768928 - Top 5 Accuracy: 0.9990167158308751
Total inference time (including transformation): 6.56 seconds


In [9]:
# Total memory usage - NOTE: This is the peak memory usage of the GPU during inference
# Please note that actual memory allocated may be higher than this value due to caching
# However, as peak memory usage gets higher, the caching will be reduced to accomodate the new memory requirements
print("Total memory usage:")
print(round(torch.cuda.max_memory_allocated() / 1024**2, 1))

Total memory usage:
1297.2
